In [51]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amex-data-integer-dtypes-parquet-format/train.parquet
/kaggle/input/amex-data-integer-dtypes-parquet-format/test.parquet
/kaggle/input/amex-default-prediction/sample_submission.csv
/kaggle/input/amex-default-prediction/train_data.csv
/kaggle/input/amex-default-prediction/test_data.csv
/kaggle/input/amex-default-prediction/train_labels.csv


**Get Input Data and Merge Data**

In [52]:
train_df = pd.read_parquet("../input/amex-data-integer-dtypes-parquet-format/train.parquet")
train_labels = pd.read_csv("../input/amex-default-prediction/train_labels.csv")
test_df = pd.read_parquet("../input/amex-data-integer-dtypes-parquet-format/test.parquet")
train_df.head()

train_data = train_df.merge(train_labels, how='inner', on="customer_ID")

**Preprocessing Data**

In [53]:
from sklearn.preprocessing import LabelEncoder
lab = LabelEncoder()
train_data['customer_ID']= lab.fit_transform(train_data['customer_ID'])
train_data = train_data.groupby('customer_ID').tail(1).set_index('customer_ID')

In [54]:
test_df['customer_ID']= lab.fit_transform(test_df['customer_ID'])
test_df = test_df.groupby('customer_ID').tail(1).set_index('customer_ID')
test_df = test_df.drop(["S_2",'D_103'],axis=1)

In [55]:
y = train_data.target
x = train_data.drop(["target"],axis=1)
x = x.drop(["S_2",'D_103'],axis=1)
print(x.shape)
print(y.shape)
print(test_df.shape)

(458913, 187)
(458913,)
(924621, 187)


In [56]:
from sklearn.preprocessing import OrdinalEncoder

# Remove columns which has missing values more than 75%
missingValueColumns = x.columns[(x.isna().sum()/len(x))*100>75]
x = x.drop(missingValueColumns, axis=1)
test_df = test_df.drop(missingValueColumns, axis=1)

# Handling Categorical variables
cols = x.columns
num_cols = x._get_numeric_data().columns
categorical_cols = list(set(cols) - set(num_cols))

ordinal_encoder = OrdinalEncoder()
x[categorical_cols]= ordinal_encoder.fit_transform(x[categorical_cols])
test_df[categorical_cols] = ordinal_encoder.transform(test_df[categorical_cols])

# Handling missing values
x = x.fillna(-123)
test_df = test_df.fillna(-123)

**Standardization**

In [57]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x = scaler.fit_transform(x)
test_df = scaler.transform(test_df)

In [58]:
x = np.array(x)
y = np.array(y)
test_x = np.array(test_df)

In [59]:
print(x.shape)
print(y.shape)
print(test_x.shape)

(458913, 176)
(458913,)
(924621, 176)


**Split Dataset into Train and Validation Set**

In [60]:
from sklearn.model_selection import train_test_split
# Splitting into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.2, random_state = 0)

**Training the Logistic Regression model on the Training set**

In [61]:
from sklearn.linear_model import LogisticRegression
logistic_classifier = LogisticRegression(random_state = 0)
logistic_classifier.fit(x_train, y_train)

y_pred_LR = logistic_classifier.predict(x_val)
print(np.concatenate((y_pred_LR.reshape(len(y_pred_LR),1), y_val.reshape(len(y_val),1)),1))

[[1 1]
 [0 0]
 [1 1]
 ...
 [0 0]
 [0 0]
 [1 1]]


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


**Making the Confusion Matrix of Logistic Regression model**

In [62]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm_LR = confusion_matrix(y_val, y_pred_LR)
print(cm_LR)
accuracy_score(y_val, y_pred_LR)

[[63591  4272]
 [ 5780 18140]]


0.8904808079927655

**Training the KNN model on the Training set**

In [63]:
#from sklearn.neighbors import KNeighborsClassifier
#kNNclassifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
#kNNclassifier.fit(x_train, y_train)

#y_pred_KNN = kNNclassifier.predict(x_val)
#print(np.concatenate((y_pred_KNN.reshape(len(y_pred_KNN),1), y_val.reshape(len(y_val),1)),1))

**Making the Confusion Matrix of KNN model**

In [64]:
#from sklearn.metrics import confusion_matrix, accuracy_score
#cm_KNN = confusion_matrix(y_val, y_pred_KNN)
#print(cm_KNN)
#accuracy_score(y_val, y_pred_KNN)

**Training the SVM model on the Training set**

In [65]:
#from sklearn.svm import SVC
#svmclassifier = SVC(kernel = 'linear', random_state = 0)
#svmclassifier.fit(x_train, y_train)

#y_pred_svm = svmclassifier.predict(x_val)
#print(np.concatenate((y_pred_svm.reshape(len(y_pred_svm),1), y_val.reshape(len(y_val),1)),1))

**Making the Confusion Matrix of SVM model**

In [66]:
#from sklearn.metrics import confusion_matrix, accuracy_score
#cm_svm = confusion_matrix(y_val, y_pred_svm)
#print(cm_svm)
#accuracy_score(y_val, y_pred_svm)

**Training XGBoost on the Training set**

In [67]:
from sklearn.metrics import confusion_matrix, accuracy_score
from xgboost import XGBClassifier

xgb_classifier = XGBClassifier()
xgb_classifier.fit(x_train, y_train)
y_pred_xgb = xgb_classifier.predict_proba(x_val)[:,1]
y_pred_xg = xgb_classifier.predict(x_val)
print(y_pred_xgb)
print(y_pred_xg)
print(y_pred_xgb.shape)
print(y_pred_xg.shape)
print(np.concatenate((y_pred_xgb.reshape(len(y_pred_xgb),1), y_val.reshape(len(y_val),1)),1))
cm_XGB = confusion_matrix(y_val, y_pred_xg)
print(cm_XGB)
accuracy_score(y_val, y_pred_xg)

[0.9810877  0.0779552  0.9798902  ... 0.00166363 0.09839445 0.82228494]
[1 0 1 ... 0 0 1]
(91783,)
(91783,)
[[0.98108768 1.        ]
 [0.0779552  0.        ]
 [0.97989023 1.        ]
 ...
 [0.00166363 0.        ]
 [0.09839445 0.        ]
 [0.82228494 1.        ]]
[[63245  4618]
 [ 4880 19040]]


0.8965167841539283

**Get predictions for the Test data**

In [68]:
predicted_test_data = xgb_classifier.predict_proba(test_x)[:,1]
print(predicted_test_data)

[0.02391857 0.00096912 0.07025836 ... 0.42286727 0.4449901  0.0451498 ]


In [69]:
sub = pd.read_csv('../input/amex-default-prediction/sample_submission.csv')

sub.head

sub['prediction']=predicted_test_data

sub.to_csv('submission.csv',index=False)
